In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import collections

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
52,tt0810819,15000000,64191523,The Danish Girl,Eddie Redmayne|Alicia Vikander|Matthias Schoen...,Tom Hooper,Find the courage to be yourself.,When Gerda Wegener asks her husband Einar to f...,120,Drama,Pretty Pictures|Working Title Films|ArtÃ©mis P...,11/27/2015,7.3,2015
1762,tt0489237,20000000,47738099,The Nanny Diaries,Scarlett Johansson|Laura Linney|Chris Evans|Pa...,Shari Springer Berman|Robert Pulcini,"A comedy about life at the top, as seen from t...",A college graduate goes to work as a nanny for...,106,Comedy|Drama|Romance,The Weinstein Company|FilmColony,8/24/2007,6.0,2007
220,tt1371150,19800000,41296320,This Is Where I Leave You,Jason Bateman|Tina Fey|Adam Driver|Rose Byrne|...,Shawn Levy,Welcome Home. Get Uncomfortable.,"When their father passes away, four grown, wor...",103,Drama|Comedy,Spring Creek Productions|21 Laps Entertainment...,9/10/2014,6.5,2014
1501,tt0463034,54000000,130431368,"You, Me and Dupree",Kate Hudson|Owen Wilson|Matt Dillon|Michael Do...,Anthony Russo|Joe Russo,Two's company. Dupree's a crowd.,After standing in as best man for his longtime...,108,Comedy|Romance,Universal Pictures|Kaplan/Perrone Entertainmen...,7/14/2006,5.3,2006
170,tt1267297,100000000,243400000,Hercules,Dwayne Johnson|Ian McShane|John Hurt|Rufus Sew...,Brett Ratner,"Before he was a legend, he was a man.","Fourteen hundred years ago, a tormented soul w...",99,Action|Adventure,Paramount Pictures|Metro-Goldwyn-Mayer (MGM)|R...,7/23/2014,5.6,2014


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# the time given in the dataset is in string format.
# So we need to change this in datetime format
# Как подсказано выше "СОЗДАТЕЛЕМ!" время (в моем случае дата) указано
# в строковом формате нужно поменять на формат времени (даты)

# Также на мой взгляд не удобными для работы являются столбцы: Cast,
# genres, production_companies - поскольку информация в них представлена
# как перечесление значений через разделитель "|", удобство работы
# возможно обеспечить преобразованием этих данных в множество.

# Поскольку преобразование данных будет идти по нескольким столбцам,
# то целесообразно написать простенькую функцию.

def convert_rec_to_set(x_rec):
    x_rec_set = [x for x in x_rec.split('|')]
    return set(x_rec_set)

# Преобразуем столбец cast и сохраняем в новый столбец 
data['cast_conv'] = (
    data.cast.apply(convert_rec_to_set)
)

# Преобразуем столбец genres и сохраняем в новый столбец
data['genres_conv'] = (
    data.genres.apply(convert_rec_to_set)
)

# Преобразуем столбец production_companies и сохраняем в новый столбец
data['company_conv'] = (
    data.production_companies.apply(convert_rec_to_set)
)

# Преобразуем столбец с датой в формат даты
data['release_date'] = pd.to_datetime(
    data.release_date, format='%m/%d/%Y'
)

# Добавляю столбик с прибыльностью фильма
data['profit'] = data.apply(lambda x: x.revenue - x.budget, axis=1)

# Также решил добавить столбец с комбинацией названия вильма и id 
# для удобства вывода ответа на большинство вопросов проекта.
data['combo_title_id'] = data.apply(
    lambda x: f'{x.original_title} ({x.imdb_id})', axis=1)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

In [5]:
display(data)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,cast_conv,genres_conv,company_conv,profit,combo_title_id
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015,"{Nick Robinson, Irrfan Khan, Vincent D'Onofrio...","{Action, Science Fiction, Adventure, Thriller}","{Legendary Pictures, Amblin Entertainment, Uni...",1363528810,Jurassic World (tt0369610)
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015,"{Tom Hardy, Josh Helman, Charlize Theron, Hugh...","{Action, Science Fiction, Adventure, Thriller}","{Village Roadshow Pictures, Kennedy Miller Pro...",228436354,Mad Max: Fury Road (tt1392190)
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015,"{Theo James, Miles Teller, Shailene Woodley, K...","{Science Fiction, Adventure, Thriller}","{NeoReel, Red Wagon Entertainment, Summit Ente...",185238201,Insurgent (tt2908446)
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,7.5,2015,"{Mark Hamill, Carrie Fisher, Adam Driver, Harr...","{Action, Science Fiction, Adventure, Fantasy}","{Lucasfilm, Truenorth Productions, Bad Robot}",1868178225,Star Wars: The Force Awakens (tt2488496)
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,7.3,2015,"{Jason Statham, Paul Walker, Dwayne Johnson, V...","{Action, Thriller, Crime}","{One Race Films, Universal Pictures, Dentsu, O...",1316249360,Furious 7 (tt2820852)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,tt0120903,75000000,157299717,X-Men,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,Adventure|Action|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,2000-07-13,6.6,2000,"{Hugh Jackman, Famke Janssen, Halle Berry, Pat...","{Action, Science Fiction, Adventure}","{Bad Hat Harry Productions, Springwood Product...",82299717,X-Men (tt0120903)
1885,tt0192255,22000000,13555988,The Little Vampire,Richard E. Grant|Jonathan Lipnicki|Jim Carter|...,Uli Edel,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,Horror|Family|Foreign,New Line Cinema,2000-10-27,6.4,2000,"{Pamela Gidley, Alice Krige, Richard E. Grant,...","{Foreign, Family, Horror}",{New Line Cinema},-8444012,The Little Vampire (tt0192255)
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,Rene Russo|Jason Alexander|Piper Perabo|Randy ...,Des McAnuff,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,Adventure|Animation|Action|Comedy|Family,Universal Pictures|Capella International|KC Me...,2000-06-30,4.0,2000,"{Randy Quaid, Robert De Niro, Rene Russo, Pipe...","{Adventure, Family, Animation, Action, Comedy}","{Tribeca Producti

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [7]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget.max() == data.budget].combo_title_id.to_list()

['Pirates of the Caribbean: On Stranger Tides (tt1298650)']

ВАРИАНТ 2

In [8]:
# можно добавлять разные варианты решения
x = data[data.budget.max() == data.budget]
x[['original_title','imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


In [9]:
# и еще один вариант
columns = ['original_title', 'imdb_id']
pd.DataFrame(data[data.budget.max() == data.budget], columns=columns)

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [11]:
data[data.runtime.max() == data.runtime].combo_title_id.to_list()

['Gods and Generals (tt0279111)']

# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [13]:
data[data.runtime.min() == data.runtime].combo_title_id.to_list()

['Winnie the Pooh (tt1449283)']

# 4. Какова средняя длительность фильмов?


In [14]:
answers['4'] = '2. 110'

In [15]:
round(data.runtime.mean(),)

110

# 5. Каково медианное значение длительности фильмов? 

In [66]:
answers['5'] = '1. 107'

In [17]:
round(data.runtime.median(),)

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [18]:
# лучше код получения столбца profit вынести в Предобработку что в начале
answers['6'] = '5. Avatar (tt0499549)'

In [19]:
data[data.profit.max() == data.profit].combo_title_id.to_list()

['Avatar (tt0499549)']

# 7. Какой фильм самый убыточный? 

In [20]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [21]:
data[data.profit.min() == data.profit].combo_title_id.to_list()

['The Lone Ranger (tt1210819)']

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [22]:
answers['8'] = '1. 1478'

In [23]:
len(data[data.profit > 0])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [24]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [25]:
data[data.release_year.values == 2008].query(
    'profit == profit.max()').combo_title_id.to_list()

['The Dark Knight (tt0468569)']

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [26]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [27]:
data_x = data.query('2012 <= release_year <= 2014')
data_x.query('profit == profit.min()').combo_title_id.to_list()

['The Lone Ranger (tt1210819)']

# 11. Какого жанра фильмов больше всего?

In [28]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

answers['11'] = '3. Drama'

In [29]:
data.genres_conv.apply(Counter).sum().most_common()

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

ВАРИАНТ 2

In [30]:
data.genres.apply(lambda x:x.split(sep='|')).explode().value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genres, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [31]:
answers['12'] = '1. Drama'

In [32]:
data[data.profit > 0].genres_conv.apply(Counter).sum().most_common()

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337),
 ('Romance', 242),
 ('Crime', 231),
 ('Family', 226),
 ('Science Fiction', 195),
 ('Fantasy', 188),
 ('Horror', 150),
 ('Animation', 120),
 ('Mystery', 119),
 ('Music', 47),
 ('History', 46),
 ('War', 41),
 ('Western', 12),
 ('Documentary', 7)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [33]:
answers['13'] = '5. Peter Jackson'

In [34]:
(
    data.groupby(['director'])['revenue']
    .sum().sort_values(ascending=False).head(1)
)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [35]:
answers['14'] = '3. Robert Rodriguez'

In [36]:
x = data[data.genres.str.contains('Action', na=False)]
(
    x.director.apply(
        lambda x:x.split(sep='|')).explode().value_counts().head(1)
    
)

Robert Rodriguez    9
Name: director, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [37]:
answers['15'] = '3. Chris Hemsworth'

In [38]:
x = data[data.release_year.values == 2012]
(
    x[['cast_conv', 'profit']].apply(
        lambda x: Counter({name: x.profit for name in x.cast_conv}
                         ),axis='columns').sum().most_common(1)
)

[('Chris Hemsworth', 1542450773)]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [39]:
answers['16'] = '3. Matt Damon'

In [40]:
(
    data[data.budget > data.budget.mean()]
    .cast_conv.apply(Counter).sum().most_common(1)
)

[('Matt Damon', 18)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [41]:
answers['17'] = '2. Action'

In [42]:
x = data[data.cast.str.contains('Nicolas Cage', na=False)]
(
    x.genres.apply(
        lambda x:x.split(sep='|')).explode().value_counts().head(1)
    
)

Action    17
Name: genres, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [43]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [44]:
x = data[data.production_companies.str
         .contains('Paramount Pictures', na=False)]
x[(x.profit.min() == x.profit)].combo_title_id.to_list()

['K-19: The Widowmaker (tt0267626)']

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [45]:
answers['19'] = '5. 2015'

In [46]:
(
    data.groupby('release_year')['revenue'].sum()
    .sort_values(ascending=False).head(1)
)

release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [47]:
answers['20'] = '1. 2014'

In [48]:
x = data[data.production_companies.str
         .contains('Warner Bros', na=False)]
(
    x.groupby('release_year')['profit'].sum()
    .sort_values(ascending=False).head(1)
)

release_year
2014    2295464519
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [49]:
answers['21'] = '4. Сентябрь'

In [50]:
monthes = []
monthes = (
    data.groupby(data.release_date.apply(lambda x: x.month))
                 .imdb_id.count().sort_values(ascending=False)
)
monthes

release_date
9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [51]:
answers['22'] = '2. 450'

In [71]:
data_monthes = data.copy()
data_monthes['rel_month'] = data.release_date.apply(lambda x: x.month)
len(data_monthes.query('6 <= rel_month <= 8'))

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [53]:
answers['23'] = '5. Peter Jackson'

In [54]:
data_monthes = data.copy()
data_monthes['rel_month'] = data.release_date.apply(lambda x: x.month)
(
    data_monthes[data_monthes.rel_month.isin(['1','2','12'])]
    .director
    .value_counts().head(1)
)

Peter Jackson    7
Name: director, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [55]:
answers['24'] = '5. Four By Two Productions'

In [56]:
data_case24 = data.copy()
data_case24['title_len'] = data_case24.original_title.apply(len)
data_case24['comp_list'] = data_case24.company_conv.apply(list)

data_case24 = (
    data_case24.explode('comp_list')
    .rename(columns={'comp_list': 'comp_exp'})
)

(
    data_case24.groupby('comp_exp')
    .title_len.max().sort_values(ascending=False).head(5)
)

comp_exp
Four By Two Productions                   83
Twentieth Century Fox Film Corporation    83
Walden Media                              62
Walt Disney                               62
Jim Henson Company, The                   59
Name: title_len, dtype: int64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [57]:
answers['25'] = '3. Midnight Picture Show'

In [58]:
data_case25 = data.copy()
data_case25['comp_list'] = data_case25.company_conv.apply(list)
data_case25 = (
    data_case25.explode('comp_list')
    .rename(columns={'comp_list': 'comp_exp'})
)
data_case25['words'] = (
    data_case25.overview.str.replace(r'\W+&\s', r' ')
    .apply(lambda x: len(str(x).split()))
)

(
    data_case25.groupby('comp_exp')
    .words.mean().sort_values(ascending=False).head(5)
)

comp_exp
Midnight Picture Show               175.0
Room 9 Entertainment                161.0
98 MPH Productions                  159.0
Heineken Branded Entertainment      159.0
Brookwell-McNamara Entertainment    156.0
Name: words, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [59]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [60]:
(
    data[data.vote_average > data.vote_average.quantile(0.99)]
    .combo_title_id
)

9                                  Inside Out (tt2096673)
34                                       Room (tt3170832)
118                              Interstellar (tt0816692)
119                   Guardians of the Galaxy (tt2015381)
125                        The Imitation Game (tt2084970)
128                                 Gone Girl (tt2267998)
138                  The Grand Budapest Hotel (tt2278388)
370                                 Inception (tt1375666)
599                           The Dark Knight (tt0468569)
872                               The Pianist (tt0253474)
1081    The Lord of the Rings: The Return of the King ...
1183                  The Wolf of Wall Street (tt0993846)
1191                         12 Years a Slave (tt2024544)
1800                                  Memento (tt0209144)
Name: combo_title_id, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [61]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [62]:
data_case27 = data.copy()
data_case27['cast_list'] = data_case27.cast_conv.apply(list)

data_case27['joined'] = (
    data_case27.cast_list.apply(lambda x: list(combinations(x, 2)))
)
data_case27 = data_case27.explode('joined')

collections.Counter(data_case27['joined']).most_common(5)

[(('Emma Watson', 'Daniel Radcliffe'), 8),
 (('Emma Watson', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Rupert Grint'), 7),
 (('Owen Wilson', 'Ben Stiller'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6)]

ВАРИАНТ 2

In [63]:
# Сдаюсь, потому как второго локаничного варианта придумать не могу.

# Submission

In [67]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [65]:
# и убедиться что ни чего не пропустил)
len(answers)

27